In [3]:
class Event:

	def __init__(self, options, self_costs, other_benefits, being_watched):
		self.options = options # possible actions
		self.self_costs = self_costs # cost of an action
		self.other_benefits = other_benefits # an action's benefits to others
		self.being_watched = being_watched # an action's ulterior benefit

In [4]:
import numpy as np

class Agent:

	def __init__(self, self_motive=1, other_motive=1, ulterior_motive=1,tau=1):
		self.self_motive = self_motive # an agent's extent of selfish motivation (stinginess)
		self.other_motive = other_motive # an agent's extent of altruistic motivation (interest in others) 
		self.ulterior_motive = ulterior_motive # an agent's extent of ulterior motivation (interest in extrinsic benefits)
		self.tau = tau # an agent's rationality 

	def GetChoiceProb(self, myevent, choice):
		utilities = [-myevent.self_costs[i]*(self.self_motive+.5)+
                     (myevent.being_watched[i]*(self.ulterior_motive))+
                     myevent.other_benefits[i]*self.other_motive for i in range(len(myevent.options))] 
        #^the utility of each event feature is weighted by an agent's relevant motive
        #add +.5 to selfish motive so that less costly options are generally preferred
		utilities = [i - max(utilities) for i in utilities] #softmax
		values = [np.exp(i/self.tau) for i in utilities] #softmax
		values = [i/sum(values) for i in values] #softmax
		return(values[choice]) #likelihood of an action being chosen over another, given a specified level of each motive

	def SetParameters(self, self_motive, other_motive, ulterior_motive):
		self.self_motive = self_motive
		self.other_motive = other_motive
		self.ulterior_motive = ulterior_motive

In [5]:
import numpy as np
import matplotlib.pyplot as plt


class Observer:

	def __init__(self, MyEvent, MyAgent):
		self.MyEvent = MyEvent
		self.MyAgent = MyAgent

	def ToM(self, ObservedChoice, samples=5000): 
		Self_Motive = [np.random.uniform(0,1) for i in range(samples)] #randomly sample different levels of each motive
		Other_Motive = [np.random.uniform(0,1) for i in range(samples)]
		Ulterior_Motive = [np.random.uniform(0,1) for i in range(samples)]
		Probability = [0] * samples
		for x in range(samples):
			self.MyAgent.SetParameters(Self_Motive[x], Other_Motive[x], Ulterior_Motive[x]) 
			Probability[x] = self.MyAgent.GetChoiceProb(self.MyEvent, ObservedChoice) #get probabilities associated with sampled motive levels
		# Normalize because this is Monte Carlo sampling
		Probability = [i/sum(Probability) for i in Probability]
		return ("estimated trait Self_Motive values",np.dot(Self_Motive,Probability),
                "estimated trait Other_Motive values",np.dot(Other_Motive,Probability),
                "estimated trait Ulterior_Motive values",np.dot(Ulterior_Motive,Probability)) #get expected value of each motive
####ignore below####
#notes from meeting:
##z score model predictions and judgements
##joint distribution, values of each and probabilities associated (asking how much they care about this thing over that thing, asking how confident you are that they are altruistic, asking how likelihood of )
    
#for plotting     
		return (plt.scatter(TraitSelfishness,TraitAltruism,c=Probability,alpha=.5),
                plt.ylabel("altruism"),
                plt.xlabel("selfishness"),
                plt.colorbar(),
                plt.show()
                ,"estimated trait Selfishness values",np.dot(TraitSelfishness,Probability),
                "estimated trait Altruism values",np.dot(TraitAltruism,Probability)) #vmin=0,vmax=1

    
    
import matplotlib as mp
#scatterplot, plot trait selfishness, and plot trait altruism
#color by probabilitiy

In [6]:
#run model#

MyEvent = Event(options=         [0, 1], # actions
                self_costs=      [1000, 0], # agent's utils on actions
                other_benefits=  [400, 0], # agent's belief of receiver's util on actions
                being_watched=   [1, 0])
MyAgent = Agent()
# Create an observer with an event and a mental model of the agent
MyObserver = Observer(MyEvent, MyAgent)
MyObserver.ToM(0) # Uncertain.

#test: ask real participants, given a chosen action
##"how selfish do you think this person is from 0-100" (not at all - extremely)
##"how altruistic do you think this person is from 0-100" (not at all - extremely)
##"how ulteriorly motivated do you think this person is from 0-100" (not at all - extremely)

('estimated trait Self_Motive values',
 0.01320375830055355,
 'estimated trait Other_Motive values',
 0.9843235019374954,
 'estimated trait Ulterior_Motive values',
 0.7017413810528961)

In [22]:
#UNINFORMATIVE: costless to help, costly not to help

result1 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [i,-(10-i)]
        MyEvent.agentbeliefs = [10-i,i]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result1.append(row)
        
result1 = np.array(result1)
new_result1 = result1.reshape(55,6)
np.savetxt('result1',new_result1)

In [6]:
for i in range(0,11):
    MyEvent.agentvalue = [i,10-i]
    MyEvent.agentbeliefs = [10-i,i]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [0, 10] unchosen: [10, 0]
event 1 | chosen: [1, 9] unchosen: [9, 1]
event 2 | chosen: [2, 8] unchosen: [8, 2]
event 3 | chosen: [3, 7] unchosen: [7, 3]
event 4 | chosen: [4, 6] unchosen: [6, 4]
event 5 | chosen: [5, 5] unchosen: [5, 5]
event 6 | chosen: [6, 4] unchosen: [4, 6]
event 7 | chosen: [7, 3] unchosen: [3, 7]
event 8 | chosen: [8, 2] unchosen: [2, 8]
event 9 | chosen: [9, 1] unchosen: [1, 9]
event 10 | chosen: [10, 0] unchosen: [0, 10]


In [7]:
#UNINFORMATIVE: no counterfactual, both options the same

result2 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [i,i]
        MyEvent.agentbeliefs = [i,i]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result2.append(row)
        
result2 = np.array(result2)
new_result2 = result2.reshape(55,6)
np.savetxt('result2',new_result2)

In [8]:
for i in range(0,11):
    MyEvent.agentvalue = [i,i]
    MyEvent.agentbeliefs = [i,i]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [0, 0] unchosen: [0, 0]
event 1 | chosen: [1, 1] unchosen: [1, 1]
event 2 | chosen: [2, 2] unchosen: [2, 2]
event 3 | chosen: [3, 3] unchosen: [3, 3]
event 4 | chosen: [4, 4] unchosen: [4, 4]
event 5 | chosen: [5, 5] unchosen: [5, 5]
event 6 | chosen: [6, 6] unchosen: [6, 6]
event 7 | chosen: [7, 7] unchosen: [7, 7]
event 8 | chosen: [8, 8] unchosen: [8, 8]
event 9 | chosen: [9, 9] unchosen: [9, 9]
event 10 | chosen: [10, 10] unchosen: [10, 10]


In [ ]:
#INFORMATIVE: costly to help, costless not to help

result3 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [i,0]
        MyEvent.agentbeliefs = [i,0]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result3.append(row)

result3 = np.array(result3)
result3
new_result3 = result3.reshape(55,6)
np.savetxt('result3',new_result3)

In [24]:
for i in range(0,11):
    MyEvent.agentvalue = [i,0,1,2,3,4,5,6,7,8,9,10]
    MyEvent.agentbeliefs = [i,0,1,2,3,4,5,6,7,8,9,10]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [0, 0] unchosen: [0, 0]
event 1 | chosen: [1, 1] unchosen: [0, 0]
event 2 | chosen: [2, 2] unchosen: [0, 0]
event 3 | chosen: [3, 3] unchosen: [0, 0]
event 4 | chosen: [4, 4] unchosen: [0, 0]
event 5 | chosen: [5, 5] unchosen: [0, 0]
event 6 | chosen: [6, 6] unchosen: [0, 0]
event 7 | chosen: [7, 7] unchosen: [0, 0]
event 8 | chosen: [8, 8] unchosen: [0, 0]
event 9 | chosen: [9, 9] unchosen: [0, 0]
event 10 | chosen: [10, 10] unchosen: [0, 0]


In [11]:
#INFORMATIVE: giving exponentially more or less

result4 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [1,5**(5-i)]
        MyEvent.agentbeliefs = [1,5**(5-i)]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result4.append(row)

result4 = np.array(result4)
new_result4 = result4.reshape(55,6)
np.savetxt('result4',new_result4)

In [12]:
for i in range(0,11):
    MyEvent.agentvalue = [1,5**(5-i)]
    MyEvent.agentbeliefs = [1,5**(5-i)]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [1, 1] unchosen: [3125, 3125]
event 1 | chosen: [1, 1] unchosen: [625, 625]
event 2 | chosen: [1, 1] unchosen: [125, 125]
event 3 | chosen: [1, 1] unchosen: [25, 25]
event 4 | chosen: [1, 1] unchosen: [5, 5]
event 5 | chosen: [1, 1] unchosen: [1, 1]
event 6 | chosen: [1, 1] unchosen: [0.2, 0.2]
event 7 | chosen: [1, 1] unchosen: [0.04, 0.04]
event 8 | chosen: [1, 1] unchosen: [0.008, 0.008]
event 9 | chosen: [1, 1] unchosen: [0.0016, 0.0016]
event 10 | chosen: [1, 1] unchosen: [0.00032, 0.00032]


In [13]:
#UNINFORMATIVE: costly or shrewdly giving with no added benefit

result5 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [2**(5-i),1]
        MyEvent.agentbeliefs = [1,1]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result5.append(row)

result5 = np.array(result5)
new_result5 = result5.reshape(55,6)
np.savetxt('result5',new_result5)

In [14]:
for i in range(0,11):
    MyEvent.agentvalue = [2**(5-i),1]
    MyEvent.agentbeliefs = [1,1]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [32, 1] unchosen: [1, 1]
event 1 | chosen: [16, 1] unchosen: [1, 1]
event 2 | chosen: [8, 1] unchosen: [1, 1]
event 3 | chosen: [4, 1] unchosen: [1, 1]
event 4 | chosen: [2, 1] unchosen: [1, 1]
event 5 | chosen: [1, 1] unchosen: [1, 1]
event 6 | chosen: [0.5, 1] unchosen: [1, 1]
event 7 | chosen: [0.25, 1] unchosen: [1, 1]
event 8 | chosen: [0.125, 1] unchosen: [1, 1]
event 9 | chosen: [0.0625, 1] unchosen: [1, 1]
event 10 | chosen: [0.03125, 1] unchosen: [1, 1]


In [15]:
#INFORMATIVE: sacrificing for the greater good

result6 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [2.5**(5-i),1]
        MyEvent.agentbeliefs = [2**(5-i),1]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result6.append(row)

result6 = np.array(result6)
new_result6 = result6.reshape(55,6)
np.savetxt('result6',new_result6)

In [16]:
for i in range(0,11):
    MyEvent.agentvalue = [2.5**(5-i),1]
    MyEvent.agentbeliefs = [2**(5-i),1]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [97.65625, 32] unchosen: [1, 1]
event 1 | chosen: [39.0625, 16] unchosen: [1, 1]
event 2 | chosen: [15.625, 8] unchosen: [1, 1]
event 3 | chosen: [6.25, 4] unchosen: [1, 1]
event 4 | chosen: [2.5, 2] unchosen: [1, 1]
event 5 | chosen: [1.0, 1] unchosen: [1, 1]
event 6 | chosen: [0.4, 0.5] unchosen: [1, 1]
event 7 | chosen: [0.16, 0.25] unchosen: [1, 1]
event 8 | chosen: [0.064, 0.125] unchosen: [1, 1]
event 9 | chosen: [0.0256, 0.0625] unchosen: [1, 1]
event 10 | chosen: [0.01024, 0.03125] unchosen: [1, 1]


In [17]:
#INFORMATIVE: sacrificing for the greater good

result7 = []

for x in range(5): #repeat the loop below 5 times
    for i in range(0,11):
        MyEvent.agentvalue = [i*4,i*1]
        MyEvent.agentbeliefs = [i*8,i*7.8]
        row = []
        row.append(MyEvent.agentvalue)
        row.append(MyEvent.agentbeliefs)
        row.append(MyObserver.ToM(0))
        result7.append(row)

result7 = np.array(result7)
new_result7 = result7.reshape(55,6)
np.savetxt('result7',new_result7)

In [18]:
for i in range(0,11):
    MyEvent.agentvalue = [i*4,i*1]
    MyEvent.agentbeliefs = [i*8,i*7.8]
    print("event",i, "| chosen:", [MyEvent.agentvalue[0],MyEvent.agentbeliefs[0]], "unchosen:",[MyEvent.agentvalue[1],MyEvent.agentbeliefs[1]])

event 0 | chosen: [0, 0] unchosen: [0, 0.0]
event 1 | chosen: [4, 8] unchosen: [1, 7.8]
event 2 | chosen: [8, 16] unchosen: [2, 15.6]
event 3 | chosen: [12, 24] unchosen: [3, 23.4]
event 4 | chosen: [16, 32] unchosen: [4, 31.2]
event 5 | chosen: [20, 40] unchosen: [5, 39.0]
event 6 | chosen: [24, 48] unchosen: [6, 46.8]
event 7 | chosen: [28, 56] unchosen: [7, 54.6]
event 8 | chosen: [32, 64] unchosen: [8, 62.4]
event 9 | chosen: [36, 72] unchosen: [9, 70.2]
event 10 | chosen: [40, 80] unchosen: [10, 78.0]
